In [1]:
import warnings
warnings.filterwarnings('ignore')

import re
import os
import string
import shutil
import json
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

import transformers
bert_tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")

bert_config = transformers.AutoConfig.from_pretrained("bert-base-uncased")

bert_config.hidden_dropout_prob = 0.2
bert_config.attention_probs_dropout_prob = 0.2
bert_model = transformers.BertModel.from_pretrained("bert-base-uncased", config=bert_config)

import datasets

In [3]:
imdb = datasets.load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
print(imdb)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [9]:
print(bert_model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.2, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.2, inplace=False)
  

In [ ]:
class imdb_for_bert(Dataset):
    
    def preprocessing_function(self, examples):
        return bert_tokenizer(
            examples['text'],
            max_length=self.max_length,
            padding='max_length',
            truncation=True)
    
    def __init__(self, text, max_length):
        self.max_length = max_length
        self.text = text
        self.tokenized_text = text.map(self.preprocessing_function)
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        input_ids = self.tokenized_text['input_ids'][idx] 
        attention_mask = self.tokenized_text['attention_mask'][idx]
        return torch.LongTensor(input_ids), torch.LongTensor(attention_mask)

In [ ]:
train_dataloader = DataLoader(imdb_for_bert(imdb['train'], 512), batch_size=16)
valid_dataloader = DataLoader(imdb_for_bert(imdb['valid'][:12500], 512), batch_size=16)
test_dataloader = DataLoader(imdb_for_bert(imdb['test'][12500:], 512), batch_size=16)

  0%|          | 0/25000 [00:00<?, ?ex/s]